In [22]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

## Notebook for finding number of schools in 2km radius

In [23]:
properties = pd.read_csv("../data/curated/input_df.csv")
properties

,URL,Rent_Price,Bedrooms,Bathrooms,Parking,Property_Type,Street,Suburb,Postcode,Distance_to_Nearest_Station_km,Cleaned_Text,Sentiment,Cleaned_HLTokens,Dominant_Topic,Dominant_Topic_Label,negative_class,positive_class,geometr_2km
0,https://www.domain.com.au/14-70-canterbury-roa...,790.0,1,1,−,Apartment / Unit / Flat,14/70 Canterbury Road,Toorak,3142,1.009717,phone enquiry d renovated victorian style unit...,0.209004,"['toorak', 'apartment', 'yard', 'unlimited', '...",1,"Beautiful, Convenient Family Homes",0.0,1.0,POLYGON ((16144528.593636421 -4557194.45220456...
1,https://www.domain.com.au/421-422-collins-stre...,475.0,1,1,−,Apartment / Unit / Flat,421/422 Collins Street,Melbourne,3000,0.777577,golden opportunity sure delight seeking fabulo...,0.271238,"['light', 'bright', 'unbeatable', 'location']",2,"Modern, Low-Maintenance Apartments",0.0,1.0,POLYGON ((16138889.938601218 -4553619.89105021...
2,https://www.domain.com.au/8-1068-lygon-street-...,730.0,3,2,2,Apartment / Unit / Flat,8/1068 Lygon Street,Carlton North,3054,1.560109,superb bedroom spacious apartment excellent na...,0.274026,"['spacious', 'bedroom', 'apartment', 'bathroom...",2,"Modern, Low-Maintenance Apartments",0.0,1.0,POLYGON ((16140046.06943675 -4548526.273504933...
3,https://www.domain.com.au/16-19-mercer-road-ar...,625.0,2,1,1,Apartment / Unit / Flat,16/19 Mercer Road,Armadale,3143,0.821527,spacious secure bedroom apartment need superb ...,0.265833,"['leafy', 'surrounds', 'central', 'lifestyle',...",0,Family Homes in Prime Locations,0.0,1.0,POLYGON ((16146240.83212016 -4558863.517405337...
4,https://www.domain.com.au/17-17-macquarie-stre...,330.0,1,1,−,Apartment / Unit / Flat,17/17 Macquarie Street,Prahran,3181,0.296351,open inspection refer unit c situated heart pr...,0.147273,"['stylish', 'student', 'accommodation', 'prahr...",3,Stunning Furnished Apartments/Townhouses,0.0,1.0,POLYGON ((16142457.594697755 -4558331.39042049...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10013,https://www.domain.com.au/10-mackinnon-court-k...,650.0,4,2,2,House,10 MacKinnon Court,Keysborough,3173,3.178492,barry plant award winning property management ...,0.252381,"['bedroom', 'bathroom', 'home']",4,Central Units for Lease,0.0,1.0,POLYGON ((16161367.904140668 -4577466.66054323...
10014,https://www.domain.com.au/10-109-manningham-st...,690.0,2,2,1,Apartment / Unit / Flat,10 109 Manningham St,Parkville,3052,0.551130,uko royal park discover premium living uko roy...,0.333571,"['park', 'living', 'speed', 'wi', 'fi', 'frien...",4,Central Units for Lease,0.0,1.0,"POLYGON ((16136731.63178592 -4549012.76962011,..."
10015,https://www.domain.com.au/2-owl-road-doreen-vi...,480.0,3,2,1,House,2 Owl Road,Doreen,3754,4.182653,modern living house located new estate laurima...,0.211616,"['turn', 'key']",3,Stunning Furnished Apartments/Townhouses,0.0,1.0,POLYGON ((16157535.752934009 -4520494.43934586...
10016,https://www.domain.com.au/89-axford-boulevard-...,620.0,4,2,2,House,89 AXFORD BOULEVARD,Wodonga,3690,281.971197,situated heart highly sought white box rise es...,0.110583,"['stunning', 'family', 'home']",4,Central Units for Lease,0.0,1.0,POLYGON ((16354918.646373019 -4320462.42301470...


In [24]:
schools = pd.read_csv("../data/landing/dv346-schoollocations2023.csv",  encoding='ISO-8859-1' )
schools

,Education_Sector,Entity_Type,School_No,School_Name,School_Type,School_Status,Address_Line_1,Address_Line_2,Address_Town,Address_State,...,Postal_Address_Line_1,Postal_Address_Line_2,Postal_Town,Postal_State,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y
0,Government,1,1,Alberton Primary School,Primary,O,21 Thomson Street,NaN,Alberton,VIC,...,21 Thomson Street,NaN,ALBERTON,VIC,3971,03 5183 2412,681,Wellington (S),146.66660,-38.61771
1,Government,1,3,Allansford and District Primary School,Primary,O,Frank Street,NaN,Allansford,VIC,...,Frank Street,NaN,ALLANSFORD,VIC,3277,03 5565 1382,673,Warrnambool (C),142.59039,-38.38628
2,Government,1,4,Avoca Primary School,Primary,O,118 Barnett Street,NaN,Avoca,VIC,...,P O Box 12,NaN,AVOCA,VIC,3467,03 5465 3176,599,Pyrenees (S),143.47565,-37.08450
3,Government,1,8,Avenel Primary School,Primary,O,40 Anderson Street,NaN,Avenel,VIC,...,40 Anderson Street,NaN,AVENEL,VIC,3664,03 5796 2264,643,Strathbogie (S),145.23472,-36.90137
4,Government,1,12,Warrandyte Primary School,Primary,O,5-11 Forbes Street,NaN,Warrandyte,VIC,...,5-11 Forbes Street,NaN,WARRANDYTE,VIC,3113,03 9844 3537,421,Manningham (C),145.21398,-37.74268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,Independent,2,2244,Plenty River College,Secondary,O,"Unit 5, 9 Danaher Drive",NaN,SOUTH MORANG,VIC,...,"Unit 5, 9 Danaher Drive",NaN,SOUTH MORANG,VIC,3752,03 9435 9060,707,Whittlesea (C),145.08148,-37.64875
2298,Catholic,2,2245,Holy Cross Catholic Primary School,Primary,O,2-14 Carlingford Rd,NaN,MICKLEHAM,VIC,...,2-14 Carlingford Rd,NaN,MICKLEHAM,VIC,3064,03 9216 4200,327,Hume (C),144.90520,-37.53046
2299,Independent,2,2246,Sidrah Gardens School,Primary,O,434-442 Belgrave-Hallam Road,NaN,NARRE WARREN NORTH,VIC,...,434-442 Belgrave-Hallam Road,NaN,NARRE WARREN NORTH,VIC,3804,03 9796 9704,161,Casey (C),145.31589,-37.97324
2300,Independent,2,2247,Mountain District Community College,Secondary,O,13-15 The Avenue,NaN,FERNTREE GULLY,VIC,...,13-15 The Avenue,NaN,FERNTREE GULLY,VIC,3156,03 9758 7859,367,Knox (C),145.29327,-37.88319


In [25]:
schools_gdf = gpd.GeoDataFrame(schools, 
                               geometry=gpd.points_from_xy(schools['X'], schools['Y']),
                               crs="EPSG:4326")

In [26]:
geom_2km_gdf = gpd.GeoDataFrame(properties, geometry=gpd.GeoSeries.from_wkt(properties['geometr_2km']), crs="EPSG:3857")

In [27]:
schools_gdf = schools_gdf.to_crs(epsg=3857)
geom_2km_gdf = geom_2km_gdf.to_crs(epsg=3857)

In [28]:
print(geom_2km_gdf.crs)

EPSG:3857


In [29]:
schools_within_2km = gpd.sjoin(schools_gdf, geom_2km_gdf, how="inner", predicate="within")
schools_count = schools_within_2km.groupby("URL").size()

In [30]:
schools_gdf

,Education_Sector,Entity_Type,School_No,School_Name,School_Type,School_Status,Address_Line_1,Address_Line_2,Address_Town,Address_State,...,Postal_Address_Line_2,Postal_Town,Postal_State,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y,geometry
0,Government,1,1,Alberton Primary School,Primary,O,21 Thomson Street,NaN,Alberton,VIC,...,NaN,ALBERTON,VIC,3971,03 5183 2412,681,Wellington (S),146.66660,-38.61771,POINT (16326851.228 -4667058.811)
1,Government,1,3,Allansford and District Primary School,Primary,O,Frank Street,NaN,Allansford,VIC,...,NaN,ALLANSFORD,VIC,3277,03 5565 1382,673,Warrnambool (C),142.59039,-38.38628,POINT (15873089.607 -4634138.859)
2,Government,1,4,Avoca Primary School,Primary,O,118 Barnett Street,NaN,Avoca,VIC,...,NaN,AVOCA,VIC,3467,03 5465 3176,599,Pyrenees (S),143.47565,-37.08450,POINT (15971636.299 -4450891.551)
3,Government,1,8,Avenel Primary School,Primary,O,40 Anderson Street,NaN,Avenel,VIC,...,NaN,AVENEL,VIC,3664,03 5796 2264,643,Strathbogie (S),145.23472,-36.90137,POINT (16167455.076 -4425367.939)
4,Government,1,12,Warrandyte Primary School,Primary,O,5-11 Forbes Street,NaN,Warrandyte,VIC,...,NaN,WARRANDYTE,VIC,3113,03 9844 3537,421,Manningham (C),145.21398,-37.74268,POINT (16165146.310 -4543138.630)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,Independent,2,2244,Plenty River College,Secondary,O,"Unit 5, 9 Danaher Drive",NaN,SOUTH MORANG,VIC,...,NaN,SOUTH MORANG,VIC,3752,03 9435 9060,707,Whittlesea (C),145.08148,-37.64875,POINT (16150396.477 -4529924.112)
2298,Catholic,2,2245,Holy Cross Catholic Primary School,Primary,O,2-14 Carlingford Rd,NaN,MICKLEHAM,VIC,...,NaN,MICKLEHAM,VIC,3064,03 9216 4200,327,Hume (C),144.90520,-37.53046,POINT (16130773.077 -4513306.265)
2299,Independent,2,2246,Sidrah Gardens School,Primary,O,434-442 Belgrave-Hallam Road,NaN,NARRE WARREN NORTH,VIC,...,NaN,NARRE WARREN NORTH,VIC,3804,03 9796 9704,161,Casey (C),145.31589,-37.97324,POINT (16176490.879 -4575646.212)
2300,Independent,2,2247,Mountain District Community College,Secondary,O,13-15 The Avenue,NaN,FERNTREE GULLY,VIC,...,NaN,FERNTREE GULLY,VIC,3156,03 9758 7859,367,Knox (C),145.29327,-37.88319,POINT (16173972.832 -4562937.594)


In [31]:
schools_count = pd.DataFrame(schools_count)
schools_count = schools_count.reset_index()
schools_count

,URL,0
0,https://www.domain.com.au/04-390-burwood-highw...,3
1,https://www.domain.com.au/07-390-burwood-highw...,3
2,https://www.domain.com.au/09-390-burwood-highw...,3
3,https://www.domain.com.au/1-04-69-flemington-r...,11
4,https://www.domain.com.au/1-1-bates-street-mal...,7
...,...,...
9699,https://www.domain.com.au/unit-64-1-wellington...,5
9700,https://www.domain.com.au/unit-9-2724-fourteen...,2
9701,https://www.domain.com.au/v803-tower-4-5-josep...,3
9702,https://www.domain.com.au/wheelers-hill-vic-31...,5


In [32]:
schools_count.columns = ["URL", "School_Counts"]

In [33]:
merged_df = pd.merge(properties, 
                     schools_count,  
                     on='URL', 
                     how='left')

merged_df

,URL,Rent_Price,Bedrooms,Bathrooms,Parking,Property_Type,Street,Suburb,Postcode,Distance_to_Nearest_Station_km,Cleaned_Text,Sentiment,Cleaned_HLTokens,Dominant_Topic,Dominant_Topic_Label,negative_class,positive_class,geometr_2km,School_Counts
0,https://www.domain.com.au/14-70-canterbury-roa...,790.0,1,1,−,Apartment / Unit / Flat,14/70 Canterbury Road,Toorak,3142,1.009717,phone enquiry d renovated victorian style unit...,0.209004,"['toorak', 'apartment', 'yard', 'unlimited', '...",1,"Beautiful, Convenient Family Homes",0.0,1.0,POLYGON ((16144528.593636421 -4557194.45220456...,9.0
1,https://www.domain.com.au/421-422-collins-stre...,475.0,1,1,−,Apartment / Unit / Flat,421/422 Collins Street,Melbourne,3000,0.777577,golden opportunity sure delight seeking fabulo...,0.271238,"['light', 'bright', 'unbeatable', 'location']",2,"Modern, Low-Maintenance Apartments",0.0,1.0,POLYGON ((16138889.938601218 -4553619.89105021...,9.0
2,https://www.domain.com.au/8-1068-lygon-street-...,730.0,3,2,2,Apartment / Unit / Flat,8/1068 Lygon Street,Carlton North,3054,1.560109,superb bedroom spacious apartment excellent na...,0.274026,"['spacious', 'bedroom', 'apartment', 'bathroom...",2,"Modern, Low-Maintenance Apartments",0.0,1.0,POLYGON ((16140046.06943675 -4548526.273504933...,11.0
3,https://www.domain.com.au/16-19-mercer-road-ar...,625.0,2,1,1,Apartment / Unit / Flat,16/19 Mercer Road,Armadale,3143,0.821527,spacious secure bedroom apartment need superb ...,0.265833,"['leafy', 'surrounds', 'central', 'lifestyle',...",0,Family Homes in Prime Locations,0.0,1.0,POLYGON ((16146240.83212016 -4558863.517405337...,8.0
4,https://www.domain.com.au/17-17-macquarie-stre...,330.0,1,1,−,Apartment / Unit / Flat,17/17 Macquarie Street,Prahran,3181,0.296351,open inspection refer unit c situated heart pr...,0.147273,"['stylish', 'student', 'accommodation', 'prahr...",3,Stunning Furnished Apartments/Townhouses,0.0,1.0,POLYGON ((16142457.594697755 -4558331.39042049...,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10013,https://www.domain.com.au/10-mackinnon-court-k...,650.0,4,2,2,House,10 MacKinnon Court,Keysborough,3173,3.178492,barry plant award winning property management ...,0.252381,"['bedroom', 'bathroom', 'home']",4,Central Units for Lease,0.0,1.0,POLYGON ((16161367.904140668 -4577466.66054323...,3.0
10014,https://www.domain.com.au/10-109-manningham-st...,690.0,2,2,1,Apartment / Unit / Flat,10 109 Manningham St,Parkville,3052,0.551130,uko royal park discover premium living uko roy...,0.333571,"['park', 'living', 'speed', 'wi', 'fi', 'frien...",4,Central Units for Lease,0.0,1.0,"POLYGON ((16136731.63178592 -4549012.76962011,...",13.0
10015,https://www.domain.com.au/2-owl-road-doreen-vi...,480.0,3,2,1,House,2 Owl Road,Doreen,3754,4.182653,modern living house located new estate laurima...,0.211616,"['turn', 'key']",3,Stunning Furnished Apartments/Townhouses,0.0,1.0,POLYGON ((16157535.752934009 -4520494.43934586...,1.0
10016,https://www.domain.com.au/89-axford-boulevard-...,620.0,4,2,2,House,89 AXFORD BOULEVARD,Wodonga,3690,281.971197,situated heart highly sought white box rise es...,0.110583,"['stunning', 'family', 'home']",4,Central Units for Lease,0.0,1.0,POLYGON ((16354918.646373019 -4320462.42301470...,1.0


In [34]:
merged_df['School_Counts'] = merged_df['School_Counts'].fillna(0)
merged_df['School_Counts'].unique()

array([ 9., 11.,  8., 14.,  1.,  7.,  6.,  2.,  0.,  5.,  4., 10., 12.,
       13.,  3., 15., 16., 17.])

In [35]:
merged_df['Rent_Price'] = pd.to_numeric(merged_df['Rent_Price'], errors='coerce')
merged_df = merged_df.dropna(subset=['Rent_Price', 'School_Counts'])

fig = px.scatter(merged_df, 
                 x='School_Counts', 
                 y='Rent_Price', 
                 title='Rent Price vs Number of Schools within 2km', 
                 labels={'School_Counts':'Number of Schools within 2km', 'Rent_Price':'Rent Price ($)'}, 
                 opacity=0.6, 
                 trendline="ols")  


fig.update_layout(
    xaxis_title="Number of Schools within 2km",
    yaxis_title="Rent Price ($)",
    template="plotly_white"
)

fig.show()


In [36]:
pearson_corr, p_value_pearson = stats.pearsonr(merged_df['School_Counts'].astype(float), merged_df['Rent_Price'])

spearman_corr, p_value_spearman = stats.spearmanr(merged_df['School_Counts'].astype(float), merged_df['Rent_Price'])

model = ols('Rent_Price ~ C(School_Counts)', data=merged_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)
print(f"Pearson Correlation: {pearson_corr}, p-value: {p_value_pearson}")
print(f"Spearman Correlation: {spearman_corr}, p-value: {p_value_spearman}")

                        sum_sq       df          F        PR(>F)
C(School_Counts)  4.667248e+06     17.0  11.848444  3.491882e-33
Residual          2.317131e+08  10000.0        NaN           NaN
Pearson Correlation: 0.09433675208719197, p-value: 3.0184759143460587e-21
Spearman Correlation: 0.10425527239074739, p-value: 1.2893330457824598e-25


In [37]:
merged_df

,URL,Rent_Price,Bedrooms,Bathrooms,Parking,Property_Type,Street,Suburb,Postcode,Distance_to_Nearest_Station_km,Cleaned_Text,Sentiment,Cleaned_HLTokens,Dominant_Topic,Dominant_Topic_Label,negative_class,positive_class,geometr_2km,School_Counts
0,https://www.domain.com.au/14-70-canterbury-roa...,790.0,1,1,−,Apartment / Unit / Flat,14/70 Canterbury Road,Toorak,3142,1.009717,phone enquiry d renovated victorian style unit...,0.209004,"['toorak', 'apartment', 'yard', 'unlimited', '...",1,"Beautiful, Convenient Family Homes",0.0,1.0,POLYGON ((16144528.593636421 -4557194.45220456...,9.0
1,https://www.domain.com.au/421-422-collins-stre...,475.0,1,1,−,Apartment / Unit / Flat,421/422 Collins Street,Melbourne,3000,0.777577,golden opportunity sure delight seeking fabulo...,0.271238,"['light', 'bright', 'unbeatable', 'location']",2,"Modern, Low-Maintenance Apartments",0.0,1.0,POLYGON ((16138889.938601218 -4553619.89105021...,9.0
2,https://www.domain.com.au/8-1068-lygon-street-...,730.0,3,2,2,Apartment / Unit / Flat,8/1068 Lygon Street,Carlton North,3054,1.560109,superb bedroom spacious apartment excellent na...,0.274026,"['spacious', 'bedroom', 'apartment', 'bathroom...",2,"Modern, Low-Maintenance Apartments",0.0,1.0,POLYGON ((16140046.06943675 -4548526.273504933...,11.0
3,https://www.domain.com.au/16-19-mercer-road-ar...,625.0,2,1,1,Apartment / Unit / Flat,16/19 Mercer Road,Armadale,3143,0.821527,spacious secure bedroom apartment need superb ...,0.265833,"['leafy', 'surrounds', 'central', 'lifestyle',...",0,Family Homes in Prime Locations,0.0,1.0,POLYGON ((16146240.83212016 -4558863.517405337...,8.0
4,https://www.domain.com.au/17-17-macquarie-stre...,330.0,1,1,−,Apartment / Unit / Flat,17/17 Macquarie Street,Prahran,3181,0.296351,open inspection refer unit c situated heart pr...,0.147273,"['stylish', 'student', 'accommodation', 'prahr...",3,Stunning Furnished Apartments/Townhouses,0.0,1.0,POLYGON ((16142457.594697755 -4558331.39042049...,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10013,https://www.domain.com.au/10-mackinnon-court-k...,650.0,4,2,2,House,10 MacKinnon Court,Keysborough,3173,3.178492,barry plant award winning property management ...,0.252381,"['bedroom', 'bathroom', 'home']",4,Central Units for Lease,0.0,1.0,POLYGON ((16161367.904140668 -4577466.66054323...,3.0
10014,https://www.domain.com.au/10-109-manningham-st...,690.0,2,2,1,Apartment / Unit / Flat,10 109 Manningham St,Parkville,3052,0.551130,uko royal park discover premium living uko roy...,0.333571,"['park', 'living', 'speed', 'wi', 'fi', 'frien...",4,Central Units for Lease,0.0,1.0,"POLYGON ((16136731.63178592 -4549012.76962011,...",13.0
10015,https://www.domain.com.au/2-owl-road-doreen-vi...,480.0,3,2,1,House,2 Owl Road,Doreen,3754,4.182653,modern living house located new estate laurima...,0.211616,"['turn', 'key']",3,Stunning Furnished Apartments/Townhouses,0.0,1.0,POLYGON ((16157535.752934009 -4520494.43934586...,1.0
10016,https://www.domain.com.au/89-axford-boulevard-...,620.0,4,2,2,House,89 AXFORD BOULEVARD,Wodonga,3690,281.971197,situated heart highly sought white box rise es...,0.110583,"['stunning', 'family', 'home']",4,Central Units for Lease,0.0,1.0,POLYGON ((16354918.646373019 -4320462.42301470...,1.0


In [38]:
merged_df.to_csv("../data/curated/input.csv", index = False)